In [17]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils
from tensorflow.keras import Sequential, layers, losses, optimizers, models
import tensorflow as tf
import os
from csv import reader
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [18]:
seed_num = 128
import random as python_random
np.random.seed(seed_num)
python_random.seed(seed_num)
tf.random.set_seed(seed_num)
from pickle import load
load_minmax_scaler = load(open('./minmax_scaler.pkl', 'rb'))

In [19]:
checkpoint_path = './temp/checkpoint.ckpt'
optimizer = optimizers.Adam()
model = Sequential()
model.add(tf.keras.layers.Dense(38, input_dim =38,activation = 'relu'))
model.add(tf.keras.layers.Dense(19,activation = 'relu'))
model.add(tf.keras.layers.Dense(19,activation = 'relu'))
model.add(tf.keras.layers.Dense(9,activation = 'relu'))
model.add(tf.keras.layers.Dense(3,  activation='softmax'))
model.compile(
    loss=sparse_categorical_crossentropy, 
    optimizer=optimizer, 
    metrics=['acc'])
model.summary()
model.load_weights(checkpoint_path)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 38)                1482      
                                                                 
 dense_11 (Dense)            (None, 19)                741       
                                                                 
 dense_12 (Dense)            (None, 19)                380       
                                                                 
 dense_13 (Dense)            (None, 9)                 180       
                                                                 
 dense_14 (Dense)            (None, 3)                 30        
                                                                 
Total params: 2,813
Trainable params: 2,813
Non-trainable params: 0
_________________________________________________________________


In [20]:
excel_data=pd.read_csv('ResultWeight_MainDimension.csv',header=None)
num_data = excel_data.to_numpy()
inx = [1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,18,20,22,24,25,26,27,28,29,30,31,32,33,34,35,36]
data1 = num_data[:,inx]
폭 = np.average(num_data[:,37:41],axis=1)
높이 = np.average(num_data[:,41:45],axis=1)
면적 = np.average(num_data[:,49:53],axis=1)
둘레 = np.average(num_data[:,53:57],axis=1)

면적둘레 = 면적/둘레
면적폭 = 면적/폭
면적높이 = 면적/높이
폭=np.reshape(폭,(num_data.shape[0],1))
높이=np.reshape(높이,(num_data.shape[0],1))
면적=np.reshape(면적,(num_data.shape[0],1))
둘레=np.reshape(둘레,(num_data.shape[0],1))
면적둘레=np.reshape(면적둘레,(num_data.shape[0],1))
면적폭=np.reshape(면적폭,(num_data.shape[0],1))
면적높이=np.reshape(면적높이,(num_data.shape[0],1))
data2 = np.concatenate((data1,폭,높이,면적,둘레,면적둘레,면적폭,면적높이,num_data[:,57:59]),axis=1)
num_data2 = data2

돼지파일명 = num_data[0:,0]
치수 = num_data2[0:,0:-2]
체형 = num_data2[0:,-1].astype('float32')
print(f'치수_shape = {치수.shape}\n체형_shape = {체형.shape}') #35개의 선별 치수 + 둘레, 폭, 높이당 면적

치수_shape = (100, 38)
체형_shape = (100,)


In [21]:
result = model.predict(load_minmax_scaler.transform(치수))
Results = np.argmax(result,axis =1)
Results = Results +1

4/4 [==============================] - 0s 1ms/step


In [22]:
cnt = 0
TF = []
for i in range(치수.shape[0]):
    if(Results[i] == 체형[i]):
        TF.append('True')
        cnt = cnt +1
    else:
        TF.append('False')
print(f"acc is {cnt/치수.shape[0]*100}")
TF = np.array(TF)
TF = np.reshape(TF,(100,1))

acc is 98.0


In [23]:
num_data

array([['6963_W184_T1_000000006963000_20210527_08_13_35_858.pcd', 870.01,
        530.01, ..., 1417.85, 184, 1],
       ['6963_W184_T1_000000006963000_20210527_08_13_35_990.pcd', 980.01,
        600.01, ..., 1296.5, 184, 1],
       ['6963_W184_T1_000000006963000_20210527_08_13_36_028.pcd',
        1000.01, 610.01, ..., 1340.92, 184, 1],
       ...,
       ['6818_W218_T2_000000006818000_20211007_08_07_40_120_245.83kg_1.4type_Clipped_c1.scd',
        850.01, 850.01, ..., 1382.29, 218, 2],
       ['6963_W184_T1_000000006963000_20210527_08_13_35_774.scd', 800.01,
        750.01, ..., 1332.6, 184, 1],
       ['6963_W184_T1_000000006963000_20210527_08_13_35_827.scd', 830.01,
        590.01, ..., 1367.49, 184, 1]], dtype=object)

In [24]:
merge = np.concatenate((num_data,np.reshape(Results,(100,1))),axis =1)

In [25]:
merge = np.concatenate((merge[:,0:1],merge[:,-2:]),axis=1)
merge = np.concatenate((merge,TF),axis = 1)
merge

array([['6963_W184_T1_000000006963000_20210527_08_13_35_858.pcd', 1, 1,
        'True'],
       ['6963_W184_T1_000000006963000_20210527_08_13_35_990.pcd', 1, 1,
        'True'],
       ['6963_W184_T1_000000006963000_20210527_08_13_36_028.pcd', 1, 1,
        'True'],
       ['6963_W184_T1_000000006963000_20210527_08_13_36_112.pcd', 1, 1,
        'True'],
       ['6963_W184_T1_000000006963000_20210527_08_13_36_159.pcd', 1, 1,
        'True'],
       ['7016_W198_T2_000000007016000_20210527_08_18_12_986.pcd', 2, 2,
        'True'],
       ['7016_W198_T2_000000007016000_20210527_08_18_13_033.pcd', 2, 2,
        'True'],
       ['7016_W198_T2_000000007016000_20210527_08_18_13_134.pcd', 2, 2,
        'True'],
       ['7016_W198_T2_000000007016000_20210527_08_18_13_239.pcd', 2, 2,
        'True'],
       ['7016_W198_T2_000000007016000_20210527_08_18_13_299.pcd', 2, 2,
        'True'],
       ['7016_W198_T2_000000007016000_20210527_08_18_13_337.pcd', 2, 2,
        'True'],
       ['7016_W198_T2

In [26]:
돼지선별 = pd.DataFrame(merge)

In [27]:
돼지선별.to_excel('돼지_100_Results.xlsx')